In [1]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [22]:

PID = pd.read_csv('Compiled/PID.csv',index_col='id_victim',parse_dates=['Date'])
Updated = pd.read_csv('Compiled/UpdatedLocations.csv',index_col='id_victim',parse_dates=['Date'])
# print(Updated)
# Updated = Updated.drop('MR_195')
PID.loc[PID.index.isin(Updated.index)]=Updated
if Updated.loc[Updated.index.isin(PID.index)==False].shape[0]>1:
    print('Changed/Missing: ',Updated.loc[Updated.index.isin(PID.index)==False])
print(PID.groupby('Check').count()['name'])
# for i,row in Updated.index)

PID = PID.reset_index(drop=False).set_index(pd.DatetimeIndex(PID.Date))


prov = 'BC'
BC = PID.loc[((PID.prov==prov))]

print()
print('Prov: '+ prov+', Total: ',BC.shape[0])
Missing = BC.loc[BC['Check']=='Yes'].sort_index(ascending=False)

print('Tasks Remaining: ',Missing.shape[0])


Check
Added      28
No        425
Yes      1175
Name: name, dtype: int64

Prov: BC, Total:  275
Tasks Remaining:  155


In [23]:
def Entries():
    city = input('Enter City/Town')
    address = input('Enter Address/Intersection')
    postal_code = input('Enter Postal Code')
    return(city,address,postal_code)
Info = ['name','age','race','Date','data_source','city_town','address_intersection','postal_code','department']
for i, row in Missing[0:1].iterrows():
    print(row[Info])
#     print(row['name','age','race','Date','data_source','department'])
#     print(row['city_town'])
    print(row['summary'])
    Skip = input('skip/add source: s/a')
    if Skip != 's':
        if Skip == 'a':
            text = input('Enter Source')
            Missing.loc[Missing.index==row['id_victim'],'extra_source']=text
            text = input('Enter Summary')
            Missing.loc[Missing.index==row['id_victim'],'summary']+='     '+text
            text = input('Enter Race')
            if text != '':
                Missing.loc[Missing.index==row['id_victim'],'race']=text
            clear_output()
            print(Missing.loc[Missing.id_victim==row['id_victim'],
                  Info].values)
            print(Missing.loc[Missing.id_victim==row['id_victim'],
                  ['summary']].values)
            
        city,address,postal_code=Entries()
        print('Entries: ',city,address,postal_code)
        keep = input('Enter/Repeat: e/r/s')
        if keep == 'e':
            Missing.loc[Missing['id_victim']==row['id_victim'],'city_town']=city
            Missing.loc[Missing['id_victim']==row['id_victim'],'address_intersection']=address
            Missing.loc[Missing['id_victim']==row['id_victim'],'postal_code']=postal_code
            Missing.loc[Missing['id_victim']==row['id_victim'],'Check']='Added'
        elif keep == 'r':
            city,address,postal_code=Entries()
            print('Entries: ',city,address,postal_code)
            keep = input('Enter: e')
            if keep == 'e':
                Missing.loc[Missing['id_victim']==row['id_victim'],'city_town']=city
                Missing.loc[Missing['id_victim']==row['id_victim'],'address_intersection']=address
                Missing.loc[Missing['id_victim']==row['id_victim'],'postal_code']=postal_code
                Missing.loc[Missing['id_victim']==row['id_victim'],'Check']='Added'
    clear_output()
    #     Unspecified
    

Missing = Missing.reset_index(drop=True).set_index(Missing['id_victim'])
Updated = Updated.append(Missing.loc[Missing['Check']=='Added'])
Updated.to_csv('Compiled/UpdatedLocations.csv')
